In [26]:
import numpy as np
from Strategy import choose_action

In [8]:
class Deck():
    def __init__(self,):
        self.cards = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']
        self.face_cards = ['J','Q','K']
    def draw(self,):
        card_drawn = np.random.choice(self.cards)
        return card_drawn

In [42]:
class BlackJack():
    def __init__(self, player):
        self.deck = Deck()
        self.hands_to_play = []
        self.dealer_public = None
        self.dealer_hidden = None
        self.bet = None
        self.hands_to_resolve = []
        self.player = player        
    def deal(self, ):
        player_hand = [self.deck.draw(), self.deck.draw()]
        self.dealer_public = self.deck.draw()
        self.dealer_hidden = self.deck.draw()
        self.hands_to_play.append(player_hand)
        
    def reset(self, bet):
        self.bet = bet
        self.deal()
        return (self.hands_to_play, self.hands_to_resolve, self.dealer_public)
    
    def hand_to_value(self, hand):
        value = 0
        aces = 0
        for card in hand:
            if card in self.deck.face_cards:
                value = value + 10
            elif card == 'A':
                value = value + 11
                aces = aces + 1
            else:
                value = value + int(card)
        while (value > 21 and aces > 0):
            value = value - 10
            aces = aces - 1
        return value
    
    
    def dealer_hit_strategy(self, dealer_hand):
        dealer_score = self.hand_to_value(dealer_hand)
        
        if(dealer_score < 17):
            dealer_hand.append(self.deck.draw())
            self.dealer_hit_strategy(dealer_hand)

        if(dealer_score in range(17, 22)):
            return dealer_score, self.check_blackjack(dealer_hand)

        if(dealer_score > 21):
            return 0, self.check_blackjack(dealer_hand)
    
    def check_blackjack(self, hand):
        value = self.hand_to_value(hand)
        if value == 21 and len(hand) == 2:
            return True
        else:
            return False
            
    def resolve(self, player_hand, dealer_score, dealer_has_blackjack):
        
        player_score = self.hand_to_value(player_hand)
        player_has_blackjack = self.check_blackjack(player_hand)
        
        if(self.is_player_bust()):
            return 0
        
        if(dealer_has_blackjack):
            if(player_has_blackjack):
                return 1
            else:
                return 0
        
        if(player_score > dealer_score):
            if player_has_blackjack:
                return 2.5
            else:
                return 2
        
        if(player_score == dealer_score):
            return 1
        
        if(player_score < dealer_score):
            return 0
           
    
    def check_player_bust(self, hand):
        value = self.hand_to_value(hand)
        if value > 21:
            return True
        else:
            return False

    def step_hand(self, action, player_hand):
        player_has_blackjack = self.check_blackjack(player_hand)
        
        if player_has_blackjack:
            return player_hand, 1
        
        else:
            if action == 'stand':
                self.hands_to_resolve.append(player_hand)
                return player_hand, 1
            
            elif action == 'hit':
                player_hand.append(deck.draw())
                is_player_bust = self.check_player_bust(player_hand)
                if is_player_bust:
                    return player_hand, 1
                else:
                    return player_hand, 0
                
            elif action == 'double':
                self.bet = self.bet * 2
                player_hand.append(deck.draw())
                is_player_bust = self.check_player_bust(player_hand)
                if is_player_bust:
                    return player_hand, 1
                else:
                    self.hands_to_resolve.append(player_hand)
                    return player_hand, 1
            
            elif action == 'split':
                assert len(player_hand) != 2, 'Player has more than two cards'
                assert player_hand[0] == player_hand[1], 'Player`s cards are different'
                self.hands_to_play.append([player_hand[0], deck.draw()])
                player_hand = [player_hand[0], deck.draw()]
                return player_hand, 0
            
    def step(self):
        while(len(self.hands_to_play) > 0):
            for player_hand in self.hands_to_play:
                done = 0
                while not done:  
                    action = self.player.choose_action(player_hand, self.dealer_public)
                    player_hand, done = self.step_hand(action, player_hand)
                    
            self.hands_to_resolve.append(player_hand)
            self.hands_to_play.remove(player_hand)   
        
        dealer_score, dealer_has_blackjack = self.dealer_hit_strategy([self.dealer_public, self.dealer_hidden])
        
        for player_hand in self.hands_to_resolve:
            reward = self.resolve(player_hand, dealer_score, dealer_has_blackjack)
            print(reward)
        

In [43]:
env = BlackJack()

In [44]:
x = env.reset(300)
x

([['4', '2']], [], '4')

In [45]:
env.dealer_hit_strategy(["Q", "6", "A"])

['Q', '6', 'A'] 17


17